In [2]:
import math
import warnings
import numpy as np
import torch.optim
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary
import random as r
import copy
from BFdataMaker import *
import time
from pretty_confusion_matrix import pp_matrix_from_data
# 示例用法
from scipy.fftpack import fft, ifft
from scipy import signal

ImportError: DLL load failed while importing _multiarray_umath: 找不到指定的模块。

In [ ]:
# 随机森林
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from time import *

test_df = pd.read_csv("t1000.csv")

def evaluate(path, model_name, num_trees=500, depth=30, num_jobs=1):
    df = pd.read_csv(path)
    y = df.values[:,0]
    x = df.values[:,1:]

    test_y = test_df.values[:,0]
    test_x = test_df.values[:,1:]

    rf = RandomForestClassifier(n_estimators=num_trees, max_depth=depth, n_jobs=num_jobs)
    start = time()
    rf.fit(x, y)
    end = time()
    elapsed = end - start
    print("Time to train model %s: %.9f seconds" % (model_name, elapsed))

    acc = np.mean(test_y == rf.predict(test_x))
    print("Model %s accuracy: %.3f" % (model_name, acc))

evaluate("t10k.csv", "10k", 500, 10, 48)    # choose your own parameter


In [ ]:
# Attention 层
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.W = nn.Linear(input_dim, 1, bias=False)

    def forward(self, input1, input2):
        # 计算注意力权重
        attention_scores = self.W(input1 + input2)
        attention_weights = F.softmax(attention_scores, dim=1)

        # 加权融合特征
        output = attention_weights * input1 + (1 - attention_weights) * input2

        return output

# 假设input1和input2是两类输入数据
input1 = torch.randn(10, 50)  # 10个样本，每个样本50维特征
input2 = torch.randn(10, 50)

# 创建注意力模块
attention = Attention(input_dim=50)

# 融合特征
output = attention(input1, input2)
print(output)

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model_time = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=5,kernel_size=9,padding=4), 
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
            nn.Conv2d(5,5,kernel_size=2,stride=2), # (40,40)
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
        )
        self.model_corp = nn.Sequential(
            nn.Conv2d(in_channels=10,out_channels=10,kernel_size=7,padding = 3),
            nn.BatchNorm2d(10,1e-6),
            nn.ReLU(),
            nn.Conv2d(10,10,kernel_size=2,stride=2),
            nn.BatchNorm2d(10,1e-6),
            nn.ReLU(),
            nn.Conv2d(in_channels=10,out_channels=15,kernel_size=5,padding = 2),
            nn.BatchNorm2d(15,1e-6),
            nn.ReLU(),
            nn.Conv2d(15,15,kernel_size=2,stride=2),
            nn.BatchNorm2d(15,1e-6),
            nn.ReLU(),
            nn.Conv2d(in_channels=15,out_channels=30,kernel_size=3,padding = 1),
            nn.BatchNorm2d(30,1e-6),
            nn.ReLU(),
            nn.Conv2d(30,30,kernel_size=2,stride=2),
            nn.Flatten(),
            # nn.Linear(750,750),
            # nn.ReLU(),
            nn.Linear(750,3),
            nn.Softmax(dim = 1)
        )
        self.model_fre = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=5,kernel_size=6), 
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
            nn.Conv2d(in_channels=5,out_channels=5,kernel_size=6), 
            nn.BatchNorm2d(5,1e-6),
            nn.ReLU(),
        )
      

    def forward(self, input_tot):
        # print(input,input.shape)
        input_time = input_tot[:,:6400]
        input_fre = input_tot[:,6400:]
        input_time = input_time.reshape(-1,1,80,80) 
        input_time = self.model_time(input_time)

        input_fre = input_fre.reshape(-1,1,50,50)
        input_fre = self.model_fre(input_fre)

        input_corp = torch.cat((input_time,input_fre),dim=1)
        input_corp = self.model_corp(input_corp)
        # print(x.shape)
        
        return input_corp
    


In [ ]:
x_current = torch.zeros(80,16,7,7)
x_voltage = torch.zeros(80,1,7,7)
print(x_current.shape, x_voltage.shape)
fusing = torch.cat((x_current,x_voltage),dim=1)
print(fusing.shape)

In [ ]:
model = Model()
print(summary(model,input_size=[(1,1, 80, 80), (1,1, 50, 50)]))


In [ ]:
import numpy as np
from scipy.fft import fft, ifft
from scipy import signal

# 生成示例信号（假设采样率为Fs）
Fs = 1000  # 采样率
T = 1/Fs  # 采样间隔
t = np.arange(0, 1, T)
signal_freq = 50  # 信号频率
signal_amplitude = 1
signal_noise = 0.2 * np.random.randn(len(t))  # 添加噪音
signal_data = signal_amplitude * np.sin(2 * np.pi * signal_freq * t) + signal_noise

# 进行FFT
signal_fft = fft(signal_data)

# 获取频率轴
freqs = np.fft.fftfreq(len(signal_data), T)
print(freqs)
# 识别主要频率分量
main_freq_index = np.argmax(np.abs(signal_fft))
main_freq = freqs[main_freq_index]
print(main_freq)
# 设计带阻滤波器
notch_freq = np.abs(main_freq)  # 带阻滤波器中心频率即为主要频率
Q = 10  # 带阻滤波器的质因数
b, a = signal.iirnotch(notch_freq, Q, Fs)

# 应用带阻滤波器
filtered_signal_fft = signal.lfilter(b, a, signal_fft)

# 逆FFT得到处理后的信号
filtered_signal = ifft(filtered_signal_fft)

# 可视化结果或进一步分析处理后的信号

In [3]:

"""
如果以6400个点做fft变换，因为采样率的原因，得到的频谱图的x轴的精度为10Hz，低于3000Hz成分只有300个点
"""

N09_M07_F10_K001_1 raw_data\K001\K001\N09_M07_F10_K001_1.mat
N09_M07_F10_KA04_1 raw_data\KA04\KA04\N09_M07_F10_KA04_1.mat


'\n如果以6400个点做fft变换，因为采样率的原因，得到的频谱图的x轴的精度为10Hz，低于3000Hz成分只有300个点\n'

In [2]:
def get_two_loader_from_1D_data(trainX,trainY,valX,valY):
    trainX,valX = torch.FloatTensor(trainX),torch.FloatTensor(valX)

    encoder = LabelEncoder()
    trainY = encoder.fit_transform(trainY.ravel())
    encoder = LabelEncoder()
    valY = encoder.fit_transform(valY.ravel())
    trainY,valY = torch.LongTensor(trainY),torch.LongTensor(valY)

    train_dataset =  torch.utils.data.TensorDataset(trainX, trainY)
    train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True, )
    test_dataset =  torch.utils.data.TensorDataset(valX, valY)
    test_loader = DataLoader(dataset=test_dataset, batch_size=len(valX), shuffle=True, )

    return train_loader,test_loader

In [ ]:
# 根据.pth创建模型实例
model_state_dict = torch.load('best_model_essay_optim.pth',map_location=torch.device('cpu'))
# 
model = Model()
# 设置模型为评估模式
model.load_state_dict(model_state_dict)
model.eval()# 将模型参数加载到模型实例中

In [10]:
tem = [1,2,3]
tem = tem.numpy()

AttributeError: 'list' object has no attribute 'numpy'

In [15]:
def train_model_process(model,train_dataloader,val_dataloader,num_epoches):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.99))
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())

    best_acc = 0.0
    train_loss_all = []
    val_loss_all = []
    train_acc_all = []
    val_acc_all = []
    since = time.time()
    

    for epoch in range(num_epoches):
        print("Epoch {}/{}".format(epoch+1,num_epoches))
        print("-"*10)

        train_loss = 0.0
        train_corrects = 0
        val_loss = 0.0
        val_corrects = 0
        train_num = 0
        val_num = 0
        total_G_mean = 0
        y_pred = []  # 存储模型的预测结果
        y_test = []  # 存储真实标签

        for step, (bx,by) in enumerate(train_dataloader):

            bx = bx.to(device)
            by = by.to(device)

            model.train()
            output = model(bx)

            # print(output)
            pre_lab = torch.argmax(output,dim = 1)
            print(pre_lab.type)
            print(by.type)
            y_pred.extend(pre_lab.numpy())  # 将预测结果添加到y_pred列表中
            y_test.extend(by.numpy())  # 将真实标签添加到y_test列表中                

            loss = criterion(output, by)
            # print('realy:',by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()* bx.size(0)
            train_corrects += torch.sum(pre_lab == by.data)
            train_num += bx.size(0)


        columns = ['health', 'fault']
        pp_matrix_from_data(np.array(y_test), np.array(y_pred), columns)  # 调用混淆矩阵可视化函数

        train_loss_all.append(train_loss/train_num)
        train_acc_all.append(train_corrects/train_num)
        # if(epoch % 5)
        y_pred = []  # 存储模型的预测结果
        y_test = []  # 存储真实标签

        total_G_mean = 0
        for step, (bx,by) in enumerate(val_dataloader):

            bx = bx.to(device)
            by = by.to(device)

            model.eval()
            output = model(bx)

            pre_lab = torch.argmax(output,dim = 1)
            loss = criterion(output, by)
            y_pred.extend(pre_lab.cpu().numpy())  # 将预测结果添加到y_pred列表中
            y_test.extend(by.cpu().numpy())  # 将真实标签添加到y_test列表中                

            val_loss += loss.item() * bx.size(0)
            val_corrects += torch.sum(pre_lab == by.data)

            val_num += bx.size(0)

            
        
        columns = ['health', 'fault']
        pp_matrix_from_data(np.array(y_test), np.array(y_pred), columns)  # 调用混淆矩阵可视化函数


        val_loss_all.append(val_loss/val_num)
        val_acc_all.append(val_corrects/val_num)

        print('Epoch {} Train Loss: {:.4f} Train Acc: {:.4f}'.format(epoch+1,train_loss_all[-1],train_acc_all[-1]))
        print('Epoch {} Test Loss: {:.4f} Test Acc: {:.4f}'.format(epoch+1,val_loss_all[-1],val_acc_all[-1]))

        if val_acc_all[-1] > best_acc:
            best_acc = val_acc_all[-1]
            best_model_wts = copy.deepcopy(model.state_dict())

        time_use = time.time() - since
        print("Time usage {:.0f}m{:.0f}s".format(time_use//60,time_use%60))

    torch.save(best_model_wts, "best_model_essay_optim.pth")
    train_process = pd.DataFrame(data = {"epoch": range(num_epoches),
                                        "train_acc_all": train_acc_all,
                                        "train_loss_all": train_loss_all,
                                        "val_acc_all": val_acc_all,
                                        "val_loss_all": val_loss_all,
                                         })
    return train_process


In [6]:
model = Model()

datadicts = {'K001':0,'KA04':1}
# for each in datadicts:
#     get_data_and_convert_to_pixels(each,datadicts[each],6400,6400,128000,output_dir="G_lp_td_fd_pixel")
data_foldername= "G_lp_td_fd_pixel"
sample_length = 8901
trainX,trainY,valX,valY = load_data_and_convert_to_grid(datadicts,0.92,data_foldername,sample_length)


N09_M07_F10_K001_1 raw_data\K001\K001\N09_M07_F10_K001_1.mat
N09_M07_F10_K001_10 raw_data\K001\K001\N09_M07_F10_K001_10.mat
N09_M07_F10_K001_11 raw_data\K001\K001\N09_M07_F10_K001_11.mat
N09_M07_F10_K001_12 raw_data\K001\K001\N09_M07_F10_K001_12.mat
N09_M07_F10_K001_13 raw_data\K001\K001\N09_M07_F10_K001_13.mat
N09_M07_F10_K001_14 raw_data\K001\K001\N09_M07_F10_K001_14.mat
N09_M07_F10_K001_15 raw_data\K001\K001\N09_M07_F10_K001_15.mat
N09_M07_F10_K001_16 raw_data\K001\K001\N09_M07_F10_K001_16.mat
N09_M07_F10_K001_17 raw_data\K001\K001\N09_M07_F10_K001_17.mat
N09_M07_F10_K001_18 raw_data\K001\K001\N09_M07_F10_K001_18.mat
N09_M07_F10_K001_19 raw_data\K001\K001\N09_M07_F10_K001_19.mat
N09_M07_F10_K001_2 raw_data\K001\K001\N09_M07_F10_K001_2.mat
N09_M07_F10_K001_20 raw_data\K001\K001\N09_M07_F10_K001_20.mat
N09_M07_F10_K001_3 raw_data\K001\K001\N09_M07_F10_K001_3.mat
N09_M07_F10_K001_4 raw_data\K001\K001\N09_M07_F10_K001_4.mat
N09_M07_F10_K001_5 raw_data\K001\K001\N09_M07_F10_K001_5.mat
N0

In [ ]:
train_loader,test_loader = get_two_loader_from_1D_data(trainX,trainY,valX,valY)

In [1]:
train_model_process(model,train_loader,test_loader,20)

NameError: name 'train_model_process' is not defined